In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
def query(payload):
    """
    Sends a payload to a TGI endpoint.
    """
    API_URL = "http://nexusraven.nexusflow.ai"
    headers = {
        "Content-Type": "application/json"
    }
    import requests
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def query_raven(prompt):
	"""
	This function sends a request to the TGI endpoint to get Raven's function call.
	This will not generate Raven's justification and reasoning for the call, to save on latency.
	"""
	import requests
	output = query({
		"inputs": prompt,
		"parameters" : {"temperature" : 0.001, "stop" : ["<bot_end>"], "do_sample" : False, "max_new_tokens" : 2048, "return_full_text" : False}})
	call = output[0]["generated_text"].replace("Call:", "").strip()
	return call

def query_raven_with_reasoning(prompt):
	"""
	This function sends a request to the TGI endpoint to get Raven's function call AND justification for the call
	"""
	import requests
	output = query({
		"inputs": prompt,
		"parameters" : {"temperature" : 0.001, "do_sample" : False, "max_new_tokens" : 2000}})
	call = output[0]["generated_text"].replace("Call:", "").strip()
	return call

def execute_sql(sql_code : str):
    import sqlite3
    
    # Connect to the database
    conn = sqlite3.connect('toy_database.db')
    cursor = conn.cursor()
    
    cursor.execute('PRAGMA table_info(toys)')
    columns = [info[1] for info in cursor.fetchall()]  # Extracting the column names
    
    # Query to select all data
    cursor.execute(sql_code)
    rows = cursor.fetchall()
    
    return_string = " ".join(columns)
    for idx, row in enumerate(rows):
        row = (idx, *row)
        return_string += "\n" + str(row)
    
    # Close the connection
    conn.close()
    return return_string

def create_random_database():
    import sqlite3
    import random

    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('toy_database.db')
    
    # Create a cursor object using the cursor() method
    cursor = conn.cursor()
    
    # Create table
    cursor.execute('''CREATE TABLE IF NOT EXISTS toys
                   (id INTEGER PRIMARY KEY, name TEXT, price REAL)''')
    
    # Define some random prefixes and suffixes for toy names
    prefixes = ['Magic', 'Super', 'Wonder', 'Mighty', 'Happy', 'Crazy']
    suffixes = ['Bear', 'Car', 'Doll', 'Train', 'Dragon', 'Robot']
    
    # Insert 100 sample data rows with random names
    for i in range(1, 101):
        toy_name = random.choice(prefixes) + ' ' + random.choice(suffixes)
        toy_price = round(random.uniform(5, 20), 2)  # Random price between 5 and 20
        cursor.execute('INSERT INTO toys (name, price) VALUES (?, ?)', (toy_name, toy_price))
    
    # Commit the transaction
    conn.commit()
    
    # Query the database
    cursor.execute('SELECT * FROM toys')
    print("Toys in database:")
    for row in cursor.fetchall():
        print(row)
    
    # Close the connection
    conn.close()

In [7]:
question = "Hey, can you tell me more about this R1 thing that was announced by Rabbit? "

no_function_calling_prompt = \
f"""
<s> [INST] {question} [/INST]
"""
query_raven(no_function_calling_prompt)

'I\'m sorry, but I don\'t have any information about a specific "R1" thing that was announced by Rabbit. It\'s possible that you may have misspoken or misunderstood the information you received. If you could provide more context or clarify your question, I would be happy to try and help you.'

In [6]:
import os


def do_web_search(full_user_prompt: str, num_results: int = 5):
    API_URL = f'{os.getenv("DLAI_TAVILY_BASE_URL", "https://api.tavily.com")}/search'
    payload = {
        "api_key": os.environ["TAVILY_API_KEY"],
        "query": full_user_prompt,
        "search_depth": "basic",
        "include_answer": False,
        "include_images": False,
        "include_raw_content": False,
        "max_results": num_results,
        "include_domains": [],
        "exclude_domains": [],
    }
    import requests

    response = requests.post(API_URL, json=payload)
    response = response.json()
    all_results = "\n\n".join(item["content"] for item in response["results"])
    return all_results

In [8]:
function_calling_prompt = """
Function:
def do_web_search(full_user_prompt : str, num_results : int = 5):
    '''
    Searches the web for the user question.
    '''

Example:
User Query: What is the oldest capital in the world?
Call: do_web_search(full_user_prompt="oldest capital")

User Query: {query}<human_end>
"""
fc_result = query_raven(function_calling_prompt.format(query=question))
print(fc_result)

do_web_search(full_user_prompt='R1 thing')


In [14]:
result = eval(fc_result)
result

"Today, Santa Monica-based AI startup rabbit inc. launched r1, a custom-built consumer AI device running on a revolutionary natural-language operating system. r1 was developed to take a leap towards an app-free online experience, by introducing an operating system that navigates all of your apps quickly and efficiently so you don't have to. r1's operating system, rabbit OS, is the first OS ever ...\n\nThe Rabbit R1 doesn't run on a conventional operating system with apps. Instead, the software is powered by AI that's been trained to use apps and services on your behalf. Rabbit founder and CEO ...\n\nFor now, the R1's feature set is much more straightforward. You can use the device to play music, get answers to questions, translate speech, take notes, summon an Uber, and a few other things.\n\nThe Rabbit R1 is an AI-powered gadget that can use your apps for you\nCES 2024: all the TVs, laptops, smart home gear, and more from the show floor\nSony brought its new PS5 colors to CES\nHow Mic

In [15]:
full_prompt = \
f"""
<s> [INST]
{result}

Use the information above to answer the following question concisely.

Question:
{question} [/INST]
"""


In [16]:
grounded_response = query_raven(full_prompt.format(question = question))


In [17]:
print (grounded_response)

The R1 is an AI-powered gadget that can use your apps for you. It's a standalone gadget about half the size of an iPhone with a 2.88-inch touchscreen, a rotating camera for taking photos and videos, and a scroll wheel / button you press to navigate around or talk to the device's built-in assistant. The R1's on-screen interface will be a series of category-based cards, for music or transportation or video chats, and Lyu says the screen mostly exists so that you can verify the model's output on your own.

The R1 also has a dedicated training mode, which you can use to teach the device how to do something, and it will supposedly be able to repeat the action on its own going forward.

The size probably isn’t bad (David Pierce has seen the R1, and says it’s not that much thicker than an iPhone), but the keynote didn’t convince me that this voice assistant is worth my time, or that allowing a “Large Action Model” access to my accounts is a good idea in terms of privacy or security.


In [18]:
create_random_database()

Toys in database:
(1, 'Mighty Robot', 11.45)
(2, 'Wonder Bear', 15.69)
(3, 'Super Bear', 19.83)
(4, 'Crazy Bear', 6.19)
(5, 'Magic Bear', 11.99)
(6, 'Magic Doll', 19.37)
(7, 'Super Train', 8.84)
(8, 'Super Robot', 17.3)
(9, 'Crazy Doll', 15.87)
(10, 'Happy Robot', 12.12)
(11, 'Crazy Doll', 16.5)
(12, 'Wonder Car', 17.74)
(13, 'Wonder Robot', 14.57)
(14, 'Magic Train', 13.98)
(15, 'Magic Bear', 8.33)
(16, 'Super Dragon', 5.31)
(17, 'Super Bear', 7.28)
(18, 'Magic Car', 16.49)
(19, 'Crazy Dragon', 6.41)
(20, 'Crazy Robot', 6.64)
(21, 'Magic Train', 12.44)
(22, 'Mighty Doll', 11.94)
(23, 'Wonder Car', 11.1)
(24, 'Crazy Robot', 6.7)
(25, 'Magic Dragon', 13.83)
(26, 'Super Train', 11.15)
(27, 'Magic Doll', 12.66)
(28, 'Mighty Robot', 11.8)
(29, 'Super Doll', 19.11)
(30, 'Happy Car', 17.18)
(31, 'Wonder Dragon', 15.5)
(32, 'Happy Dragon', 12.82)
(33, 'Crazy Bear', 15.1)
(34, 'Super Bear', 15.6)
(35, 'Crazy Car', 13.93)
(36, 'Happy Robot', 17.44)
(37, 'Crazy Robot', 8.18)
(38, 'Happy Train', 

In [19]:
question = "What is the most expensive item we currently sell?"

In [20]:
schema = \
"""
CREATE TABLE IF NOT EXISTS toys (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
);
"""

raven_prompt = \
f'''
Function:
def execute_sql(sql_code : str):
  """
  Runs sql code for a company internal database
  """

Schema: {schema}
User Query: {question}
'''

output = query_raven(raven_prompt)
print (f"LLM's function call: {output}")
database_result = eval(output)

LLM's function call: execute_sql(sql_code='SELECT name, price FROM toys ORDER BY price DESC LIMIT 1')


In [21]:
full_prompt = \
f"""
<s> [INST]
{database_result}

Use the information above to answer the following question concisely.

Question:
{question} [/INST]
"""
grounded_response = query_raven(full_prompt)
print (grounded_response)

The most expensive item we currently sell is Super Bear, which costs 19.83.


In [22]:
import sqlite3
import random

# Internal database name setting
DB_NAME = 'toy_database.db'

# Connect to the database
def connect_db():
    return sqlite3.connect(DB_NAME)

In [23]:
# List all toys
def list_all_toys():
    with connect_db() as conn:
        cursor = conn.execute('SELECT * FROM toys')
        return cursor.fetchall()


In [24]:
# Find toy by name prefix
def find_toy_by_prefix(prefix):
    with connect_db() as conn:
        query = 'SELECT * FROM toys WHERE name LIKE ?'
        cursor = conn.execute(query, (prefix + '%',))
        return cursor.fetchall()


In [25]:
# Find toys in a price range
def find_toys_in_price_range(low_price, high_price):
    with connect_db() as conn:
        query = 'SELECT * FROM toys WHERE price BETWEEN ? AND ?'
        cursor = conn.execute(query, (low_price, high_price))
        return cursor.fetchall()

In [26]:
# Get a random selection of toys
def get_random_toys(count=5):
    with connect_db() as conn:
        cursor = conn.execute('SELECT * FROM toys')
        all_toys = cursor.fetchall()
        return random.sample(all_toys, min(count, len(all_toys)))

In [27]:
# Function to get the most expensive toy
def get_most_expensive_toy(count=1):
    with connect_db() as conn:
        cursor = conn.execute(f'SELECT * FROM toys ORDER BY price DESC LIMIT {count}')
        return cursor.fetchone()


In [28]:
# Function to get the cheapest toy
def get_cheapest_toy(count=1):
    with connect_db() as conn:
        cursor = conn.execute('SELECT * FROM toys ORDER BY price ASC LIMIT {count}')
        return cursor.fetchone()

In [29]:
raven_prompt = \
f'''
Function:
def list_all_toys():
    """
    Retrieves a list of all toys from the database. This function does not take any parameters.
    Returns: A list of tuples, where each tuple represents a toy with all its attributes (id, name, price).
    """

Function:
def find_toy_by_prefix(prefix):
    """
    Searches for and retrieves toys whose names start with a specified prefix.
    Parameters:
    - prefix (str): The prefix to search for in toy names.
    Returns: A list of tuples, where each tuple represents a toy that matches the prefix criteria.
    """

Function:
def find_toys_in_price_range(low_price, high_price):
    """
    Finds and returns toys within a specified price range.
    Parameters:
    - low_price (float): The lower bound of the price range.
    - high_price (float): The upper bound of the price range.
    Returns: A list of tuples, each representing a toy whose price falls within the specified range.
    """

Function:
def get_random_toys():
    """
    Selects and returns a random set of toys from the database, simulating a "featured toys" list.

    Returns: A list of tuples, each representing a randomly selected toy. The number of toys returned is up to the specified count.
    """

Function:
def get_most_expensive_toy(count : int):
    """
    Retrieves the most expensive toy from the database.
    This function does not take any parameters.

    Returns: A tuple representing the most expensive toy, including its id, name, and price.
    """

Function:
def get_cheapest_toy(count : int):
    """
    Finds and retrieves the cheapest toy in the database.
    This function does not take any parameters.

    Returns: A tuple representing the cheapest toy, including its id, name, and price.
    """

User Query: {question}<human_end>

'''

output = query_raven(raven_prompt)
print (output)
results = eval(output)

get_most_expensive_toy(count=1)


In [30]:
full_prompt = \
f"""
<s> [INST]
{database_result}

Use the information above to answer the following question in a single sentence.

Question:
{question} [/INST]
"""
grounded_response = query_raven(full_prompt)
print (grounded_response)

The most expensive item we currently sell is Super Bear, which costs 19.83.
